In [ ]:
#accessing Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#libraries

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [ ]:
#functions

def number_of_labels(number_labels):
  if number_labels == 4:

    #keeping 4 labels
    #(changing label range)
    train_labels[train_labels == 1] = 0
    valid_labels[valid_labels == 1] = 0
    train_labels[train_labels == 2] = 1
    valid_labels[valid_labels == 2] = 1
    train_labels[train_labels == 3] = 2
    valid_labels[valid_labels == 3] = 2
    train_labels[train_labels == 4] = 3
    valid_labels[valid_labels == 4] = 3

  elif number_labels == 3:

    #changing from 4 to 3 labels
    #(merging partially deforested and partially preserved)
    train_labels[train_labels == 3] = 2
    valid_labels[valid_labels == 3] = 2
    train_labels[train_labels == 1] = 0
    valid_labels[valid_labels == 1] = 0
    train_labels[train_labels == 2] = 1
    valid_labels[valid_labels == 2] = 1
    train_labels[train_labels == 4] = 2
    valid_labels[valid_labels == 4] = 2

  elif number_labels == 2:

    #changing from 4 to 2 labels
    #(merging partially deforested, partially preserved
    #and totally deforested)
    train_labels[train_labels == 3] = 1
    valid_labels[valid_labels == 3] = 1
    train_labels[train_labels == 2] = 1
    valid_labels[valid_labels == 2] = 1
    train_labels[train_labels == 1] = 0
    valid_labels[valid_labels == 1] = 0
    train_labels[train_labels == 4] = 1
    valid_labels[valid_labels == 4] = 1

In [ ]:
#loading datasets

train_images = np.load("/content/drive/MyDrive/TCC2/npy_files/"
                        + "train_images.npy")
valid_images = np.load("/content/drive/MyDrive/TCC2/npy_files/"
                        + "valid_images.npy")
train_labels = np.load("/content/drive/MyDrive/TCC2/npy_files/"
                        + "train_labels.npy")
valid_labels = np.load("/content/drive/MyDrive/TCC2/npy_files/"
                        + "valid_labels.npy")

In [ ]:
#choosing the number of labels

number_labels = int(input("Number of labels? "))
number_epochs = int(input("Number of epochs? "))

number_of_labels(number_labels)

In [ ]:
#creating convolutional model

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu',
                        input_shape=(363, 358, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(number_labels, activation='softmax'))

In [ ]:
#compiling the model

model.compile(optimizer = 'Adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
#training the model

history = model.fit(x = train_images,
                    y = train_labels,
                    validation_data = (valid_images, valid_labels),
                    batch_size = 10,
                    epochs = number_epochs,
                    verbose = 2)

In [ ]:
#evaluating the model

test_loss, test_acc = model.evaluate(valid_images, valid_labels,
                                     verbose=2)

In [ ]:
#plotting the chart

plt.figure().gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.plot(list(np.arange(1,number_epochs+1)),
         history.history['accuracy'], label='Treino')
plt.plot(list(np.arange(1,number_epochs+1)),
         history.history['val_accuracy'], label = 'Validação')
plt.axhline(test_acc, color='r',
            label=str(round(test_acc*100, 2)) + "%", linewidth=0.5)
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.title('Acurária do Treino/Validação por Época')
plt.ylim([0.5, 1])
plt.xlim([1, number_epochs])
plt.legend(loc='lower right')
plt.plot()

In [ ]:
#saving the model

model.save('/content/drive/MyDrive/TCC2/models/cnn_' + str(number_labels)
            + '_Labels_' + str(number_epochs) + '_epochs.h5')

In [ ]:
#model architecture summary

model.summary()